<a href="https://colab.research.google.com/github/dishant2809/dcgan/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dishantdeshmukh","key":"fdf6836dcb9115e37e4ac374455bbbde"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d birdy654/football-players-and-staff-faces

 76% 34.0M/44.6M [00:00<00:00, 78.5MB/s]
100% 44.6M/44.6M [00:00<00:00, 146MB/s] 


In [5]:
from zipfile import ZipFile
file_name = 'football-players-and-staff-faces.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import cv2
from skimage.transform import resize
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Conv2D,Reshape,Dense,Conv2DTranspose,BatchNormalization,LeakyReLU,Add,Activation,Input
from tensorflow.keras.optimizers import SGD, Adam

In [7]:
img_h = 64
img_w = 64
img_c = 3
d = img_h*img_w

In [8]:
def discriminator_model(img_size):
    
    i = Input(shape=(img_size,))
    x = Dense(1024,activation=LeakyReLU())(i)
    
    x = Dense(512,activation=LeakyReLU())(x)
    
    x = Dense(256,activation=LeakyReLU())(x)
    
    x = Dense(1,activation='sigmoid')(x)
    
    model = Model(i,x)
    return model


In [9]:

latent_dim = 100

def generator_model():
    
    i = Input(shape = (latent_dim,))
    x = Dense(256,activation=LeakyReLU())(i)
    x = BatchNormalization()(x)
    
    x = Dense(512,activation=LeakyReLU())(x)
    x = BatchNormalization()(x)
    
    x = Dense(1024,activation=LeakyReLU())(x)
    x = BatchNormalization()(x)
    
    x = Dense(d,activation='tanh')(x)
   
    
    model = Model(i,x)

    return model

In [10]:
folder_path = '/content/football'
list_img = os.listdir(folder_path)

In [11]:

img = []
for i in list_img:
    a = cv2.imread(folder_path+'/'+i)
    a = cv2.cvtColor(a,cv2.COLOR_RGB2GRAY)
#    a = a / 255.0 * 2 - 1
    a = cv2.resize(a,(img_h,img_w))
#    a = a.reshape(-1,d)
    img.append(a)
    

img = np.array(img)
img = img / 255.0 * 2 - 1
img = img.reshape(-1,d)


In [12]:

discriminator = discriminator_model(d)
discriminator.compile( loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

generator = generator_model()


In [13]:
discriminator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4096)]            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              4195328   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 4,851,713
Trainable params: 4,851,713
Non-trainable params: 0
_________________________________________________________________


In [14]:
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096

In [15]:
z = Input(shape=(latent_dim,))
z.shape

TensorShape([None, 100])

In [16]:
gen = generator(z)
gen

<KerasTensor: shape=(None, 4096) dtype=float32 (created by layer 'model_1')>

In [17]:
discriminator.trainable = False

In [18]:
fake_pred = discriminator(gen)

In [19]:
combined_model = Model(z, fake_pred)

combined_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [20]:
batch_size = 64
epochs = 20000
sample_period = 1000

In [21]:
zeros = np.zeros(batch_size)
ones = np.ones(batch_size)

d_losses = []
g_losses = []

if not os.path.exists('gan_img'):
    os.makedirs('gan_img')

In [22]:
def sample_images(epoch):
  rows, cols = 2, 2
  noise = np.random.randn(rows * cols, latent_dim)
  imgs = generator.predict(noise)

  # Rescale images 0 - 1
  imgs = 0.5 * imgs + 0.5

  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(img_h, img_w), cmap='gray')
      axs[i,j].axis('off')
      idx += 1
  fig.savefig("gan_img/%d.png" % epoch)
  plt.close()

In [ ]:

for epoch in range(epochs):
      
    idx = np.random.randint(0, img.shape[0], batch_size)
    real_imgs = img[idx]
    
    
    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator.predict(noise)
    
    
    plt.imshow(fake_imgs)
    
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)

    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc  = 0.5 * (d_acc_real + d_acc_fake)
    
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones)
    
    
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones)
    
    
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    
    if epoch % 100 == 0:
      print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, \
        d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
    
    if epoch % sample_period == 0:
      sample_images(epoch)

epoch: 1/20000, d_loss: 0.82,         d_acc: 0.43, g_loss: 0.64
epoch: 101/20000, d_loss: 0.12,         d_acc: 0.96, g_loss: 3.86
epoch: 201/20000, d_loss: 0.55,         d_acc: 0.70, g_loss: 1.22
epoch: 301/20000, d_loss: 0.58,         d_acc: 0.72, g_loss: 1.30
epoch: 401/20000, d_loss: 0.63,         d_acc: 0.62, g_loss: 0.97
epoch: 501/20000, d_loss: 0.67,         d_acc: 0.60, g_loss: 0.88
epoch: 601/20000, d_loss: 0.76,         d_acc: 0.34, g_loss: 0.80
epoch: 701/20000, d_loss: 0.69,         d_acc: 0.51, g_loss: 0.77
epoch: 801/20000, d_loss: 0.70,         d_acc: 0.48, g_loss: 0.77
epoch: 901/20000, d_loss: 0.70,         d_acc: 0.50, g_loss: 0.82
epoch: 1001/20000, d_loss: 0.69,         d_acc: 0.49, g_loss: 0.77
epoch: 1101/20000, d_loss: 0.73,         d_acc: 0.50, g_loss: 0.80
epoch: 1201/20000, d_loss: 0.69,         d_acc: 0.45, g_loss: 0.83
epoch: 1301/20000, d_loss: 0.71,         d_acc: 0.45, g_loss: 0.80
epoch: 1401/20000, d_loss: 0.70,         d_acc: 0.38, g_loss: 0.80
epoch: 